In [1]:
import json
import os
import sys
from PIL import Image
from tqdm import tqdm

# --- 사용자 설정 경로 ---
# 100% 전체 데이터셋 JSON 경로
JSON_PATH = "/nas/datahub/llava-v1.5-instruct/llava_v1_5_mix665k.json"

# 이미지 폴더의 루트 경로
IMAGE_ROOT = "/nas/datahub/llava-v1.5-instruct"

# OCR_VQA 폴더 이름 (필터링용)
TARGET_FOLDER = "ocr_vqa"

# 확인해볼 후보 확장자 리스트 (우선순위 순서대로)
CANDIDATE_EXTS = ['.png', '.gif', '.bmp', '.webp', '.tiff', '.jpeg']
# ---------------------

def convert_any_to_jpg_if_missing():
    print(f"Loading JSON data from {JSON_PATH}...")
    try:
        with open(JSON_PATH, 'r') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Error: JSON file not found at {JSON_PATH}")
        sys.exit(1)

    print(f"Total samples in JSON: {len(data)}")
    
    converted_count = 0
    skipped_count = 0
    ignored_folder_count = 0
    error_count = 0
    missing_all_count = 0

    print(f"Starting check and conversion process (Target Folder: {TARGET_FOLDER})...")
    
    for item in tqdm(data, desc="Processing Images"):
        # 'image' 키가 있는 경우만 처리
        if 'image' in item:
            relative_path_jpg = item['image']

            # 1. OCR_VQA 폴더에 있는 파일인지 확인 (아니면 건너뜀)
            if not relative_path_jpg.startswith(TARGET_FOLDER):
                ignored_folder_count += 1
                continue
            
            # 2. 타겟 JPG 파일 경로 설정
            full_path_jpg = os.path.join(IMAGE_ROOT, relative_path_jpg)
            
            # 3. 이미 JPG가 존재하면 건너뜀
            if os.path.exists(full_path_jpg):
                skipped_count += 1
                continue
            
            # 4. JPG가 없으면, 다른 포맷의 원본 파일이 있는지 탐색
            # 확장자를 제외한 파일 경로 추출
            file_base_path, _ = os.path.splitext(full_path_jpg)
            
            found_source = False
            
            for ext in CANDIDATE_EXTS:
                full_path_source = file_base_path + ext
                
                if os.path.exists(full_path_source):
                    try:
                        # 5. 원본(PNG/GIF 등) -> JPG 변환 및 저장
                        with Image.open(full_path_source) as img:
                            # RGBA(투명 배경)나 P(팔레트) 모드를 RGB로 변환
                            rgb_im = img.convert('RGB')
                            rgb_im.save(full_path_jpg, quality=95)
                            
                            converted_count += 1
                            found_source = True
                            # 변환 성공했으면 다른 확장자는 더 이상 찾지 않고 루프 탈출
                            break
                    except Exception as e:
                        print(f"\n[Error] Failed to convert {full_path_source}: {e}")
                        error_count += 1
                        found_source = True # 파일은 찾았으나 에러난 경우도 찾은 걸로 처리하여 missing 카운트 방지
                        break
            
            if not found_source:
                # 후보 확장자 리스트를 다 뒤져도 파일이 없는 경우
                missing_all_count += 1

    print("\n" + "="*40)
    print("       Conversion Summary       ")
    print("="*40)
    print(f"⏭️  Skipped (Not in '{TARGET_FOLDER}'): {ignored_folder_count}")
    print(f"✅ Already existed (Skipped)   : {skipped_count}")
    print(f"🔄 Converted (Other -> JPG)    : {converted_count}")
    print(f"❌ Conversion Errors           : {error_count}")
    print(f"❓ Missing (No candidate found): {missing_all_count}")
    print("="*40)

if __name__ == "__main__":
    # PIL 이미지 용량 제한 해제
    Image.MAX_IMAGE_PIXELS = None
    convert_any_to_jpg_if_missing()

Loading JSON data from /nas/datahub/llava-v1.5-instruct/llava_v1_5_mix665k.json...
Total samples in JSON: 665298
Starting check and conversion process (Target Folder: ocr_vqa)...


Processing Images: 100%|██████████| 665298/665298 [13:07<00:00, 844.65it/s]    



       Conversion Summary       
⏭️  Skipped (Not in 'ocr_vqa'): 544610
✅ Already existed (Skipped)   : 79998
🔄 Converted (Other -> JPG)    : 2
❌ Conversion Errors           : 0
❓ Missing (No candidate found): 0
